### Get Cloud outage data into dataframe

First import the json and define column names and there mapping to json

In [33]:
import json
import pandas as pd

COLS = {"id": "id",    
        "start": "begin",
        "end": "end",
        "severity": "severity",
        "status": "status_impact",
        "location": ("currently_affected_locations", "previously_affected_locations"),
        "service_name": "service_name",
        "number_affected_products": "affected_products",
        "country": None,
    }

with open("../json-data/incidents.json") as file:
    incidents_json = json.load(file)

Create data frame and copy over the data

In [111]:
data_centers = pd.read_csv("../data/data-centers.csv", dtype={
    'city': 'string','country':'string','id':'string'})
data_centers = data_centers.set_index("id")
# print(data_centers)

In [114]:
df = pd.DataFrame(data=[], columns=COLS.keys())

for incident in incidents_json:
    locations = [x["id"] for x in incident.get(COLS["location"][0]) + incident.get(COLS["location"][1], None)]
    # print([item for item in locations if item not in data_centers["id"].to_list()])
    countries = [data_centers.at[id, "country"] for id in locations]
    
    row = {
        "id": incident.get(COLS["id"], None),
        "start": incident.get(COLS["start"], None),
        "end": incident.get(COLS["end"], None),
        "severity": incident.get(COLS["severity"], None),
        "location": locations,
        "country": countries,
        "service_name": incident.get(COLS["service_name"]),
        "status": incident.get(COLS["status"], None),
        "number_affected_products": len(incident.get(COLS["number_affected_products"], []))
    }
    df.loc[len(df)] = row


In [117]:
print(df.head())

                     id                      start                        end  \
0  PVePoauUpR6sgi79f7V1  2023-09-17T03:43:18+00:00  2023-09-17T04:51:53+00:00   
1  EBxyHQgEPnbM3Syag5yL  2023-09-13T23:05:13+00:00                       None   
2  MGpGgLVRbkS7P84D9mrZ  2023-09-13T08:03:41+00:00  2023-09-13T08:53:36+00:00   
3  71zrZxw3PRkiNYhM1JsZ  2023-09-13T06:46:00+00:00  2023-09-13T10:32:00+00:00   
4  mRFjk3tKPSUJfxA6okrv  2023-09-12T15:14:26+00:00  2023-09-12T21:26:57+00:00   

  severity               status  \
0      low  SERVICE_INFORMATION   
1      low  SERVICE_INFORMATION   
2      low  SERVICE_INFORMATION   
3     high       SERVICE_OUTAGE   
4      low  SERVICE_INFORMATION   

                                            location  \
0  [asia-east1, asia-east2, asia-northeast1, asia...   
1  [asia-east1, asia-east2, asia-northeast1, asia...   
2                                      [us-central1]   
3                                      [us-central1]   
4                     

Analyze the data

In [119]:
res = df[df['severity'].isin(["high"])]
print(res[['severity', 'start', 'end', 'country']])
print(res.shape)

    severity                      start                        end  \
3       high  2023-09-13T06:46:00+00:00  2023-09-13T10:32:00+00:00   
6       high  2023-09-11T11:00:00+00:00  2023-09-11T19:25:00+00:00   
29      high  2023-08-14T23:11:00+00:00  2023-08-15T18:22:00+00:00   
45      high  2023-07-28T17:17:00+00:00  2023-07-28T22:52:00+00:00   
144     high  2023-04-26T08:27:34+00:00  2023-04-26T09:18:36+00:00   
147     high  2023-04-26T06:45:00+00:00  2023-04-26T23:07:00+00:00   
148     high  2023-04-26T02:00:00+00:00  2023-04-27T00:05:00+00:00   
203     high  2023-03-09T09:30:00+00:00  2023-03-09T10:24:00+00:00   
209     high  2023-03-03T19:18:00+00:00  2023-03-03T20:45:00+00:00   
213     high  2023-02-27T12:58:00+00:00  2023-02-27T13:11:00+00:00   
276     high  2022-12-09T12:50:00+00:00  2022-12-09T15:55:00+00:00   
282     high  2022-12-02T01:30:00+00:00  2022-12-03T04:50:00+00:00   
283     high  2022-12-01T20:20:00+00:00  2022-12-01T21:36:00+00:00   
290     high  2022-1